# Medoid Compositing
*Seasonal Composite Landsat TM/ETM+ Images Using the Medoid (a Multi-Dimensional Median), Neil Flood, 2013, doi:10.3390/rs5126481*

In [1]:
import ee
from geetools import ui, tools, composite, cloud_mask, indices

## Build a collection

In [2]:
p = ee.Geometry.Point(-72, -42)

In [3]:
col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
        .filterBounds(p).filterDate('2017-01-01', '2017-12-01')\
        .map(cloud_mask.landsat8SR_pixelQA())\
        .map(indices.ndvi('B5', 'B4'))\
        .limit(7)

In [4]:
ui.eprint(col.size())

7



## Other simple composites to compare

In [5]:
max_ndvi = col.qualityMosaic('ndvi')

In [6]:
mosaic = col.mosaic()

## Medoid

In [7]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [8]:
medoid = composite.medoid(col, bands=bands)

## Show on Map

In [9]:
Map = ui.Map()
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [10]:
vis = {'bands':['B5', 'B6','B4'], 'min':0, 'max':5000}

In [11]:
Map.addLayer(p)
Map.centerObject(p)

In [12]:
Map.addLayer(max_ndvi, vis, 'max NDVI')

In [13]:
Map.addLayer(mosaic, vis, 'simply Mosaic')

In [14]:
Map.addLayer(medoid, vis, 'Medoid')

## Extract data from images and compute locally to compare

Extract medoid values in point

In [15]:
medoid_values = tools.image.get_value(medoid.select(bands), p, scale=30, side='client')

In [16]:
medoid_values

{'B2': 77, 'B3': 188, 'B4': 105, 'B5': 2500, 'B6': 691, 'B7': 224}

List of values

In [17]:
medoid_values_list = [val for _, val in medoid_values.items()]

In [18]:
medoid_values_list

[224, 105, 188, 691, 2500, 77]

Extract values at point in each image of the collection

In [19]:
col_values = tools.imagecollection.get_values(col.select(bands), p, scale=30, side='client')

Get bandnames

In [20]:
col_key_list = []
for _, d in col_values.items():
    keys = []
    for k, v in d.items():
        keys.append(k)        
    col_key_list.append(keys)

In [21]:
col_key_list

[['B7', 'B4', 'B3', 'B6', 'B5', 'B2'],
 ['B7', 'B4', 'B3', 'B6', 'B5', 'B2'],
 ['B7', 'B4', 'B3', 'B6', 'B5', 'B2'],
 ['B7', 'B4', 'B3', 'B6', 'B5', 'B2'],
 ['B7', 'B4', 'B3', 'B6', 'B5', 'B2'],
 ['B7', 'B4', 'B3', 'B6', 'B5', 'B2'],
 ['B7', 'B4', 'B3', 'B6', 'B5', 'B2']]

Get values as a list

In [22]:
col_values_list = []
for _, d in col_values.items():
    values = []
    for _, v in d.items():
        if v:
            values.append(v)
        else:
            values.append(0)
    col_values_list.append(values)

In [23]:
col_values_list

[[0, 0, 0, 0, 0, 0],
 [287.0, 143.0, 272.0, 870.0, 3168.0, 112.0],
 [224.0, 127.0, 200.0, 698.0, 2445.0, 83.0],
 [245.0, 107.0, 193.0, 720.0, 2465.0, 87.0],
 [259.0, 120.0, 210.0, 813.0, 2717.0, 90.0],
 [224.0, 105.0, 188.0, 691.0, 2500.0, 77.0],
 [307.0, 159.0, 290.0, 928.0, 3142.0, 107.0]]

## Medoid Method locally

In [24]:
def local_medoid(values):
    from copy import copy
    import math

    def distance(arr1, arr2):
        zipped = zip(arr1, arr2)
        accum = 0
        for a, b in zipped:
            calc = (a-b)*(a-b)
            accum += calc
        return math.sqrt(accum)

    def med(values):
        results = {}
        for i, val in enumerate(values):
            val = list(val)
            cop = copy(values)
            cop = [list(a) for a in cop]
            cop.remove(val)
            dist = 0
            for r in cop:
                r = list(r)
                d = distance(val, r)
                dist += d
            results[i] = dist

        return results
    
    def getmin(d):
        minval = min(d.values())
        for k, v in d.items():
            if v == minval:
                return k
    
    values = med(values)
    min_value = getmin(values)
    
    # return the index of the minimized sum as first argument, and all options as second
    return min_value, values

## Compute medoid locally and compare

In [25]:
local = local_medoid(col_values_list)

In [26]:
local

(5,
 {0: 17251.127189606195,
  1: 6022.895734074118,
  2: 4461.227704323097,
  3: 4399.422196724633,
  4: 4593.00466857401,
  5: 4380.0310171598885,
  6: 5996.407209928899})

Get the values that correspond to the medoid

In [27]:
min_values = col_values_list[local[0]]

In [28]:
min_values

[224.0, 105.0, 188.0, 691.0, 2500.0, 77.0]

Match bands with values

In [29]:
local_medoid = dict(zip(col_key_list[0], min_values))

In [30]:
local_medoid

{'B2': 77.0, 'B3': 188.0, 'B4': 105.0, 'B5': 2500.0, 'B6': 691.0, 'B7': 224.0}

In [31]:
medoid_values

{'B2': 77, 'B3': 188, 'B4': 105, 'B5': 2500, 'B6': 691, 'B7': 224}

## Finally, compare values from medoid mosaic against locally computed medoid (from images values)

In [32]:
medoid_values == local_medoid

True